In [1]:
!pip install fairseq sentencepiece sacremoses transformers tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 82.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 663.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.7/273.7 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11170788 sha256=5b3f33220d7a2bebdd47e34bf4902e26e385edc3b0e76bc1aabc45d1746a2d20
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Cr

In [2]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import BertConfig, BertTokenizer, BertModel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
config = BertConfig.from_pretrained("bert-base-multilingual-cased")
model = BertModel.from_pretrained("bert-base-multilingual-cased")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
weights = model.get_input_embeddings().weight.detach().numpy()
vocab = tokenizer.get_vocab()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import pickle
base_data_dir = "./data/tokenized_data"
for data_file in os.listdir(base_data_dir):
# for data_file in ['test.en', 'test.fa']:
    print("Process", data_file)
    with open(os.path.join(base_data_dir, data_file), 'r') as file:
        data = file.read().splitlines()
        tokenized_input = tokenizer.batch_encode_plus(data, add_special_tokens=True, max_length=16, padding='max_length', truncation=True)
    lines = ""
    for ids in tokenized_input['input_ids']:
        tokens = tokenizer.convert_ids_to_tokens(ids)
        lines += " ".join(tokens)
        lines += "\n"
    with open(os.path.join("data/encoded_data/", data_file), 'w') as f:
        f.write(lines)

In [14]:
word_embeddings = []
with open(os.path.join("./data/bert_weights.txt"), 'w') as f:
    for word, index in vocab.items():
        embedding = weights[index]
        line = word  + " " + " ".join(map(str, embedding.tolist()[:512]))
        f.write(f"{line}\n")

In [ ]:
!mkdir -p ./data/data_bin_bert

!fairseq-preprocess --source-lang en --target-lang fa \
  --trainpref ./data/encoded_data/train \
  --validpref ./data/encoded_data/valid \
  --testpref ./data/encoded_data/test \
  --nwordstgt 40000 \
  --nwordssrc 40000 \
  --joined-dictionary \
  --destdir ./data/data_bin_bert/

2023-05-31 22:58:39.671107: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-31 22:58:42 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-05-31 22:58:42 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, s

In [5]:
!fairseq-train \
    "./data/data_bin_bert/" \
    --arch lstm --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.0025 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.25 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.2 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --fp16 --memory-efficient-fp16 \
    --max-epoch 5 \
    --encoder-embed-path ./data/bert_weights.txt \
    --decoder-embed-path ./data/bert_weights.txt \
    --share-all-embeddings \
    --source-lang en \
    --target-lang fa \
    --save-dir ./data/data_bin_bert/checkpoints/ \
    --tensorboard-logdir ./data/data_bin_bert/logs

2023-06-01 21:01:21.927559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-01 21:01:23 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-06-01 21:01:29 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': './data/data_bin_bert/logs', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_siz